# Creating the GEM-PRO

In [1]:
import sys
import logging
import os.path as op
import pandas as pd
from ssbio.pipeline.gempro import GEMPRO

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] %(levelname)s: %(message)s', datefmt="%Y-%m-%d %H:%M")
handler.setFormatter(formatter)
logger.handlers = [handler]

In [2]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Specify the folders and model
GEM_NAME = 'iML1515_GP'
ROOT_DIR = '.'
GEM_FILE = 'iML1515_GP/model/iML1515.json'

In [4]:
# Create the GEM-PRO project
my_gempro = GEMPRO(gem_name=GEM_NAME, root_dir=ROOT_DIR, gem_file_path=GEM_FILE, gem_file_type='json', pdb_file_type='mmtf')

[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: iML1515: loaded model
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: 2719: number of reactions
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: 2229: number of reactions linked to a gene
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: 1515: number of genes (excluding spontaneous)
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: 1919: number of metabolites
[2017-09-05 11:19] [ssbio.pipeline.gempro] WARNING: IMPORTANT: All Gene objects have been transformed into GenePro objects, and will be for any new ones
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: ./iML1515_GP: GEM-PRO project location
[2017-09-05 11:19] [ssbio.pipeline.gempro] INFO: 1515: number of genes


In [5]:
my_gempro.kegg_mapping_and_metadata(kegg_organism_code='eco')
my_gempro.uniprot_mapping_and_metadata(model_gene_source='ENSEMBLGENOME_ID')
my_gempro.manual_uniprot_mapping({'b4104':'P16683', 'b2092':'P69831'})
my_gempro.set_representative_sequence()

[2017-07-20 16:16] [root] WARNING: status is not ok with Not Found
[2017-07-20 16:16] [ssbio.databases.kegg] WARNING: eco:b2092: no sequence file available
[2017-07-20 16:16] [root] WARNING: status is not ok with Not Found
[2017-07-20 16:16] [ssbio.databases.kegg] WARNING: eco:b4104: no sequence file available
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: 1513/1515: number of genes mapped to KEGG
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: Completed ID mapping --> KEGG. See the "df_kegg_metadata" attribute for a summary dataframe.
[2017-07-20 16:16] [root] INFO: getUserAgent: Begin
[2017-07-20 16:16] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.18.1
[2017-07-20 16:16] [root] INFO: getUserAgent: End


[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: 1513/1515: number of genes mapped to UniProt
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: Completed ID mapping --> UniProt. See the "df_uniprot_metadata" attribute for a summary dataframe.


[2017-07-20 16:16] [ssbio.pipeline.gempro] ERROR: b4104, P16683: unable to complete web request
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: Completed manual ID mapping --> UniProt. See the "df_uniprot_metadata" attribute for a summary dataframe.
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: 1515/1515: number of genes with a representative sequence
[2017-07-20 16:16] [ssbio.pipeline.gempro] INFO: See the "df_representative_sequences" attribute for a summary dataframe.


HTTP Error 300: Multiple Choices




In [6]:
my_gempro.map_uniprot_to_pdb()
my_gempro.blast_seqs_to_pdb(seq_ident_cutoff=.98, all_genes=False)

[2017-07-20 12:22] [ssbio.pipeline.gempro] INFO: Mapping UniProt IDs --> PDB IDs...
[2017-07-20 12:22] [root] INFO: getUserAgent: Begin
[2017-07-20 12:22] [root] INFO: getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.18.1
[2017-07-20 12:22] [root] INFO: getUserAgent: End
[2017-07-20 12:22] [ssbio.pipeline.gempro] INFO: 671/1515: number of genes with at least one experimental structure
[2017-07-20 12:22] [ssbio.pipeline.gempro] INFO: Completed UniProt --> best PDB mapping. See the "df_pdb_ranking" attribute for a summary dataframe.


[2017-07-20 12:23] [ssbio.core.protein] WARNING: b4104: no representative sequence loaded, cannot BLAST
[2017-07-20 12:23] [ssbio.pipeline.gempro] INFO: Completed sequence --> PDB BLAST. See the "df_pdb_blast" attribute for a summary dataframe.
[2017-07-20 12:23] [ssbio.pipeline.gempro] INFO: 14: number of genes with additional structures added from BLAST


In [7]:
# Creating manual mapping dictionary for ECOLI I-TASSER models
homology_models = '/home/nathan/projects_archive/homology_models/ECOLI/zhang/'
homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/zhang_data/160804-ZHANG_INFO.csv')
tmp = homology_models_df[['zhang_id','model_file','m_gene']].drop_duplicates()
tmp = tmp[pd.notnull(tmp.m_gene)]

homology_model_dict = {}

for i,r in tmp.iterrows():
    homology_model_dict[r['m_gene']] = {r['zhang_id']: {'model_file':op.join(homology_models, r['model_file']),
                                                        'file_type':'pdb'}}
    
my_gempro.get_manual_homology_models(homology_model_dict)

[2017-07-20 12:51] [ssbio.pipeline.gempro] INFO: Updated homology model information for 1512 genes.


In [8]:
# Creating manual mapping dictionary for ECOLI SUNPRO models
homology_models = '/home/nathan/projects_archive/homology_models/ECOLI/sunpro/'
homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/sunpro_data/160609-SUNPRO_INFO.csv')
tmp = homology_models_df[['sunpro_id','model_file','m_gene']].drop_duplicates()
tmp = tmp[pd.notnull(tmp.m_gene)]

homology_model_dict = {}

for i,r in tmp.iterrows():
    homology_model_dict[r['m_gene']] = {r['sunpro_id']: {'model_file':op.join(homology_models, r['model_file']),
                                                         'file_type':'pdb'}}
    
my_gempro.get_manual_homology_models(homology_model_dict)

[2017-07-20 12:51] [ssbio.pipeline.gempro] INFO: Updated homology model information for 1413 genes.


In [9]:
my_gempro.set_representative_structure()

[2017-07-20 12:55] [ssbio.core.protein] ERROR: 5hbo: unable to parse structure file as mmtf. Falling back to mmCIF format.
[2017-07-20 13:05] [ssbio.pipeline.gempro] INFO: 1513/1515: number of genes with a representative structure
[2017-07-20 13:05] [ssbio.pipeline.gempro] INFO: See the "df_representative_structures" attribute for a summary dataframe.


# Adding domain information

In [10]:
domains = pd.read_csv('/home/nathan/projects/iML1515_GP/iML1515_GP/data/iML1515_domains.csv', index_col=0)

In [11]:
domains.head()

,fatcat_domain,u_uniprot_acc,pdb
0,d1hqsa_,P08200,1AI2
1,d1hqsa_,P08200,1AI3
2,d1hqsa_,P08200,1BL5
3,d1hqsa_,P08200,1CW1
4,d1hqsa_,P08200,1CW4


In [12]:
for g in my_gempro.genes_with_a_representative_sequence:
    for s in g.protein.sequences:
        if s.uniprot in domains.u_uniprot_acc.tolist():
            s.domains = domains[domains.u_uniprot_acc == s.uniprot]['fatcat_domain'].unique()
        else:
            s.domains = None
    
    if g.protein.representative_sequence.uniprot in domains.u_uniprot_acc.tolist():
        g.protein.representative_sequence.domains = domains[domains.u_uniprot_acc == s.uniprot]['fatcat_domain'].unique().tolist()
    else:
        g.protein.representative_sequence.domains = None
        
    for t in g.protein.structures:
        if t.is_experimental:
            if t.id.upper() in domains.pdb.tolist():
                t.domains = domains[domains.pdb == t.id.upper()]['fatcat_domain'].unique().tolist()
            else:
                t.domains = None
        else:
            t.domains = None
    
    if g.protein.representative_structure:
        if g.protein.representative_structure.is_experimental:
            if g.protein.representative_structure.original_pdb_id.upper() in domains.pdb.tolist():
                g.protein.representative_structure.domains = domains[domains.pdb == g.protein.representative_structure.original_pdb_id.upper()]['fatcat_domain'].unique().tolist()
            else:
                g.protein.representative_structure.domains = None
        else:
            g.protein.representative_structure.domains = None

# Saving

In [13]:
my_gempro.save_json('/home/nathan/projects/iML1515_GP/iML1515_GP/model/iML1515_GP.json', compression=False)

[2017-07-20 13:17] [root] WARNING: json-tricks: numpy scalar serialization is experimental and may work differently in future versions
[2017-07-20 13:18] [ssbio.core.io] INFO: Saved <class 'ssbio.pipeline.gempro.GEMPRO'> (id: iML1515_GP) to /home/nathan/projects/iML1515_GP/iML1515_GP/model/iML1515_GP2.json


In [14]:
my_gempro.save_json('/home/nathan/projects/iML1515_GP/iML1515_GP.json.gz', compression=True)

[2017-07-20 13:20] [ssbio.core.io] INFO: Saved <class 'ssbio.pipeline.gempro.GEMPRO'> (id: iML1515_GP) to /home/nathan/projects/iML1515_GP/iML1515_GP2.json.gz


In [15]:
my_gempro.save_pickle(outfile='/home/nathan/projects/iML1515_GP/iML1515_GP/model/iML1515_GP.pckl')

# Recreating the original dataframe

In [1]:
import ssbio.core.io

In [2]:
my_gempro = ssbio.core.io.load_pickle('/home/nathan/projects/iML1515_GP/iML1515_GP/model/iML1515_GP.pckl')

In [19]:
df_uniprot = my_gempro.df_uniprot_metadata
df_kegg = my_gempro.df_kegg_metadata
df_repseq = my_gempro.df_representative_sequences
df_pdbblast = my_gempro.df_pdb_blast
df_pdbrank = my_gempro.df_pdb_ranking
df_repstruct = my_gempro.df_representative_structures

In [20]:
logger.setLevel(logging.CRITICAL)
df_pdbmetadata = my_gempro.df_pdb_metadata

In [21]:
it_homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/zhang_data/160804-ZHANG_INFO.csv')
sp_homology_models_df = pd.read_csv('/home/nathan/projects_archive/homology_models/ECOLI/sunpro_data/160609-SUNPRO_INFO.csv')

In [26]:
pre_df = []
df_gempro = pd.DataFrame()
for rxn in my_gempro.model.reactions:
    for gene in rxn.genes:
        if gene.id == 's0001':
            continue
        gene_dict = {}
        gene_dict['m_reaction'] = rxn.id
        gene_dict['m_gene_reaction_rule'] = rxn.gene_reaction_rule
        gene_dict['m_metabolites'] = ';'.join([x.id for x in rxn.metabolites])
        gene_dict['m_subsystem'] = rxn.subsystem
        gene_dict['m_gene'] = gene.id
        
        gene_dict['seq_uniprot'] = df_repseq.loc[gene.id, 'uniprot']
        gene_dict['seq_kegg'] = df_repseq.loc[gene.id, 'kegg']
        gene_dict['seq_mapped_pdbs'] = df_repseq.loc[gene.id, 'pdbs']
        if my_gempro.genes.get_by_id(gene.id).protein.representative_sequence.domains:
            gene_dict['seq_domains'] = ';'.join(my_gempro.genes.get_by_id(gene.id).protein.representative_sequence.domains)
        gene_dict['seq_len'] = df_repseq.loc[gene.id, 'seq_len']
        gene_dict['seq_file'] = df_repseq.loc[gene.id, 'sequence_file']
        
        if my_gempro.genes.get_by_id(gene.id).protein.representative_structure:
            repstruct = df_repstruct.loc[gene.id, 'id']
            ispdb = df_repstruct.loc[gene.id, 'is_experimental']
            gene_dict['struct_id'] = repstruct
            gene_dict['struct_is_experimental'] = ispdb
            gene_dict['struct_percent_seq_cov'] = gene.protein.representative_chain_seq_coverage
            if ispdb:
                pdb_id = repstruct.split('-')[0]
                sliced = df_pdbmetadata[df_pdbmetadata.pdb_id == pdb_id]
                gene_dict['struct_pdb'] = sliced.loc[gene.id, 'pdb_id']
                gene_dict['struct_resolution'] = sliced.loc[gene.id, 'resolution']
                gene_dict['struct_chemicals'] = sliced.loc[gene.id, 'chemicals']
                if my_gempro.genes.get_by_id(gene.id).protein.representative_structure.domains:
                    gene_dict['struct_domains'] = ';'.join(my_gempro.genes.get_by_id(gene.id).protein.representative_structure.domains)
            else:
                if '_ECOLI' not in repstruct:
                    ident = repstruct.split('-')[0]
                    sliced = sp_homology_models_df[sp_homology_models_df.sunpro_id == ident]
                else:
                    ident = repstruct.split('-')[0]
                    sliced = it_homology_models_df[it_homology_models_df.zhang_id == ident]
                    template = sliced.i_template.values[0]
                    gene_dict['struct_homology_template'] = template

            gene_dict['struct_file'] = df_repstruct.loc[gene.id, 'structure_file']
        
        pre_df.append(gene_dict)

In [27]:
final_df = pd.DataFrame(pre_df, columns=['m_reaction','m_gene_reaction_rule','m_metabolites','m_subsystem','m_gene','seq_uniprot','seq_kegg','seq_mapped_pdbs','seq_domains','seq_len','seq_file','struct_id','struct_is_experimental','struct_percent_seq_cov','struct_pdb','struct_resolution','struct_chemicals','struct_domains','struct_homology_template','struct_file']).sort_values(by='m_reaction').reset_index(drop=True).set_index('m_reaction')

In [28]:
final_df.to_csv(op.join(my_gempro.data_dir, 'iML1515_GP.csv'))

In [29]:
final_df

,m_gene_reaction_rule,m_metabolites,m_subsystem,m_gene,seq_uniprot,seq_kegg,seq_mapped_pdbs,seq_domains,seq_len,seq_file,struct_id,struct_is_experimental,struct_percent_seq_cov,struct_pdb,struct_resolution,struct_chemicals,struct_domains,struct_homology_template,struct_file
m_reaction,,,,,,,,,,,,,,,,,,,
12PPDRtex,(b0241 or b0929 or b1377 or b2215),12ppd__R_e;12ppd__R_p,"Transport, Outer Membrane Porin",b0241,P02932,ecj:JW0231;eco:b0241,1PHO,PDP:5FVNAa,351,P02932.fasta,PHOE_ECOLI-X,False,100.0,NaN,NaN,NaN,NaN,1pho,PHOE_ECOLI_model1_clean-X_clean.pdb
12PPDRtex,(b0241 or b0929 or b1377 or b2215),12ppd__R_e;12ppd__R_p,"Transport, Outer Membrane Porin",b1377,P77747,ecj:JW1371;eco:b1377,NaN,NaN,377,P77747.fasta,OMPN_ECOLI-X,False,100.0,NaN,NaN,NaN,NaN,2ixx,OMPN_ECOLI_model1_clean-X_clean.pdb
12PPDRtex,(b0241 or b0929 or b1377 or b2215),12ppd__R_e;12ppd__R_p,"Transport, Outer Membrane Porin",b0929,P02931,ecj:JW0912;eco:b0929,4JFB;4GCS;2OMF;1GFP;4LSH;1MPF;1OPF;4GCQ;1HXX;2...,PDP:5FVNAa,362,P02931.fasta,2zfg-A,True,93.6,2zfg,1.590,MG;C8E,PDP:5FVNAa,NaN,2zfg-A_clean.pdb
12PPDRtex,(b0241 or b0929 or b1377 or b2215),12ppd__R_e;12ppd__R_p,"Transport, Outer Membrane Porin",b2215,P06996,ecj:JW2203;eco:b2215,2J1N;2ZLE;2J4U;4A8D;3NB3,PDP:5FVNAa,367,P06996.fasta,2j1n-A,True,94.3,2j1n,2.000,MG;CL;D12,PDP:5FVNAa,NaN,2j1n-A_clean.pdb
12PPDStex,(b0241 or b0929 or b1377 or b2215),12ppd__S_p;12ppd__S_e,"Transport, Outer Membrane Porin",b0929,P02931,ecj:JW0912;eco:b0929,4JFB;4GCS;2OMF;1GFP;4LSH;1MPF;1OPF;4GCQ;1HXX;2...,PDP:5FVNAa,362,P02931.fasta,2zfg-A,True,93.6,2zfg,1.590,MG;C8E,PDP:5FVNAa,NaN,2zfg-A_clean.pdb
12PPDStex,(b0241 or b0929 or b1377 or b2215),12ppd__S_p;12ppd__S_e,"Transport, Outer Membrane Porin",b2215,P06996,ecj:JW2203;eco:b2215,2J1N;2ZLE;2J4U;4A8D;3NB3,PDP:5FVNAa,367,P06996.fasta,2j1n-A,True,94.3,2j1n,2.000,MG;CL;D12,PDP:5FVNAa,NaN,2j1n-A_clean.pdb
12PPDStex,(b0241 or b0929 or b1377 or b2215),12ppd__S_p;12ppd__S_e,"Transport, Outer Membrane Porin",b0241,P02932,ecj:JW0231;eco:b0241,1PHO,PDP:5FVNAa,351,P02932.fasta,PHOE_ECOLI-X,False,100.0,NaN,NaN,NaN,NaN,1pho,PHOE_ECOLI_model1_clean-X_clean.pdb
12PPDStex,(b0241 or b0929 or b1377 or b2215),12ppd__S_p;12ppd__S_e,"Transport, Outer Membrane Porin",b1377,P77747,ecj:JW1371;eco:b1377,NaN,NaN,377,P77747.fasta,OMPN_ECOLI-X,False,100.0,NaN,NaN,NaN,NaN,2ixx,OMPN_ECOLI_model1_clean-X_clean.pdb
13PPDH2,b3011,13ppd_c;3hppnl_c;nadp_c;h_c;nadph_c,Alternate Carbon Metabolism,b3011,Q46856,ecj:JW2978;eco:b3011,1OJ7;4QGS,PDP:4QGSAa;PDP:4QGSAb;PDP:4QGSAc,387,Q46856.fasta,4qgs-A,True,97.9,4qgs,1.700,ZN;CL,PDP:4QGSAa;PDP:4QGSAb;PDP:4QGSAc,NaN,4qgs-A_clean.pdb


******************